#### 访问数据是使用本书所介绍的这些工具的第一步。我会着重介绍 pandas 的数据输入与输出，虽然别的库中也有不少以此为目的的工具。输入输出通常可以划分为几个大类:读取文本文件和其他更高效的磁盘存储格 式，加载数据库中的数据，利用 Web API 操作网络资源。


#### pandas 提供了一些用于将表格型数据读取为 DataFrame 对象的函数。表 6-1 对 它们进行了总结，其中 read_csv 和 read_table 可能会是你今后用得最多的。


read_csv  从文件、URL、文件型对象中加载带分隔符的数据、默认分隔符为逗号
read_table 从文件、URL、文件型对象中加载带分隔符的数据
read_fwf  读取定宽列函数

#### 我将大致介绍一下这些函数在将文本数据转换为 DataFrame 时所用到的一些技术。
这些函数的选项可以划分为以下几个大类:
1 索引:将一个或多个列当做返回的 DataFrame 处理，以及是否从文件、 用户获取列名。
2 类型推断和数据转换:包括用户定义值的转换、和自定义的缺失值标记 列表等。
3 日期解析:包括组合功能，比如将分散在多个列中的日期时间信息组合 成结果中的单个列。
4 迭代:支持对大文件进行逐块迭代。
5 不规整数据问题:跳过一些行、页脚、注释或其他一些不重要的东西
   (比如由成千上万个逗号隔开的数值数据)。


#### 因为工作中实际碰到的数据可能十分混乱，一些数据加载函数(尤其是 read_csv)的选项逐渐变得复杂起来。面对不同的参数，感到头痛很正常 (read_csv 有超过 50 个参数)。pandas 文档有这些参数的例子，如果你感到 阅读某个文件很难，可以通过相似的足够多的例子找到正确的参数。


#### 其中一些函数，比如 pandas.read_csv，有类型推断功能，因为列数据的类型 不属于数据类型。也就是说，你不需要指定列的类型到底是数值、整数、布尔 值，还是字符串。其它的数据格式，如 HDF5、Feather 和 msgpack，会在格式 中存储数据类型。


#### 日期和其他自定义类型的处理需要多花点工夫才行。首先我们来看一个以逗号 分隔的(CSV)文本文件:


In [7]:
import pandas as pd 
df = pd.read_csv('example.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 我们还可以使用 read_table，并指定分隔符:

In [8]:
pd.read_table('example.csv',sep=',')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### 并不是所有文件都有标题行。看看下面这个文件:

In [11]:
pd.read_csv('example.csv',header= None)

,0,1,2,3,4
0,a,b,c,d,message
1,1,2,3,4,hello
2,5,6,7,8,world
3,9,10,11,12,foo


#### 假设你希望将 message 列做成 DataFrame 的索引。你可以明确表示要将该列放 到索引 4 的位置上，也可以通过 index_col 参数指定"message":


In [13]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('example.csv', names=names,index_col='message')

,a,b,c,d
message,,,,
message,a,b,c,d
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


#### 如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可:

In [14]:
pursed = pd.read_csv('example.csv',index_col = ['key1','key2'])
pursed

ValueError: Index key1 invalid

#### 有些情况下，有些表格可能不是用固定的分隔符去分隔字段的(比如空白符或其他模式)。有些表格可能不是用固定的分隔符去分隔字段的(比如空白符或其他模式来分隔字段)。看看下面这个文本文件:


#### 虽然可以手动对数据进行规整，这里的字段是被数量不同的空白字符间隔开 的。这种情况下，你可以传递一个正则表达式作为 read_table 的分隔符。可以 用正则表达式表达为\s+，于是有有:


#### 缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有 (空字符串)，要么用某个标记值表示。默认情况下，pandas 会用一组经常出 现的标记值进行识别，比如 NA 及 NULL:


In [15]:
#pd.isnull(result) 
# 可以使用isnull 函数对数据的中的缺失值进行标注   是NA的话 显示的是 True 否则是 False


#### na_values 可以用一个列表或集合的字符串表示缺失值:

In [16]:
#  result = pd.read_csv('example.csv',na_value =['NULL'])
# 可以对缺失值直接在读取文件的时候进行标识

In [17]:
#  read_csv 和 read_table 函数的参数
#path 表示文件系统的位置 URL 文件对象的字符串
#sep或delimiter 用于对行中各字段进行拆分的字符序列或正则表达式
# header 用作列名的行号，默认为0，如果没有header行就应该设置为None
# index_col 用作行索引的列编号或列名，可以使单个名称/数字或由多个名称/数字组成的列表
# names  用于结果的列名列表，结合header = None
#  skiprows  需要忽略的行数，或者需要跳过的行号列表（从0开始）
# na_values 一组用于替换NA的值
# commet 用于将注释信息从行尾分出去的字符

#### 在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，你可能只想读取文件的一小部分或逐块对文件进行迭代。


#### 在看大文件之前，我们先设置 pandas 显示地更紧些:

In [18]:
#pd.options.display.max_rows=10  只显示10行

#### 要逐块读取文件，可以指定 chunksize(行数):

In [19]:
chunker = pd.read_csv('example.csv',chunksize=2)
chunker

#### read_csv 所返回的这个 TextParser 对象使你可以根据 chunksize 对文件进行 逐块迭代。比如说，我们可以迭代处理 ex6.csv，将值计数聚合到"key"列中， 如下所示:


In [20]:
#chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
#tot = pd.Series([])
#for piece in chunker:
#    tot = tot.add(piece['key'].value_counts(), fill_value=0)
#tot = tot.sort_values(ascending=False)

#### 数据也可以被输出为分隔符格式的文本。我们再来看看之前读过的一个 CSV 文 件:


#### 利用 DataFrame 的 to_csv 方法，我们可以将数据写到一个以逗号分隔的文件 中:


In [25]:
!cat examples/out.csv
0,one,1,2,3.0,4,
1,two,5,6,8,world
2,three,9,10,11.0,12,foo


cat: examples/out.csv: No such file or directory


NameError: name 'one' is not defined

#### 如何将(一个或一组)JSON 对象转换为 DataFrame 或其他便于分析的数据结构 就由你决定了。最简单方便的方式是:向 DataFrame 构造器传入一个字典的列 表(就是原先的 JSON 对象)，并选取数据字段的子集:


#### pandas.read_json 可以自动将特别格式的 JSON 数据集转换为 Series 或 DataFrame。例如:


In [29]:
!cat examples/example.json[{"a": 1, "b": 2, "c": 3},{"a": 4, "b": 5, "c": 6},{"a": 7, "b": 8, "c": 9}]

cat: examples/example.json[{a:: No such file or directory
cat: 1,: No such file or directory
cat: b:: No such file or directory
cat: 2,: No such file or directory
cat: c:: No such file or directory
cat: 3},{a:: No such file or directory
cat: 4,: No such file or directory
cat: b:: No such file or directory
cat: 5,: No such file or directory
cat: c:: No such file or directory
cat: 6},{a:: No such file or directory
cat: 7,: No such file or directory
cat: b:: No such file or directory
cat: 8,: No such file or directory
cat: c:: No such file or directory
cat: 9}]: No such file or directory


#### pandas.read_json 的默认选项假设 JSON 数组中的每个对象是表格中的一行:

In [31]:
data = pd.read_json('examples/example.json')
data

ValueError: Expected object or value

#### 许多网站都有一些通过 JSON 或其他格式提供数据的公共 API。通过 Python 访 问这些 API 的办法有不少。一个简单易用的办法(推荐)是 requests 包 

#### 为了搜索最新的 30 个 GitHub 上的 pandas 主题，我们可以发一个 HTTP GET 请 求，使用 requests 扩展库:


#### 在商业场景下，大多数数据可能不是存储在文本或 Excel 文件中。基于 SQL 的 关系型数据库(如 SQL Server、PostgreSQL 和 MySQL 等)使用非常广泛，其它 一些数据库也很流行。数据库的选择通常取决于性能、数据完整性以及应用程 序的伸缩性需求。


#### 将数据从 SQL 加载到 DataFrame 的过程很简单，此外 pandas 还有一些能够简化 该过程的函数。例如，我将使用 SQLite 数据库(通过 Python 内置的 sqlite3 驱动器):
